In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pydrake.all import (AbstractValue, Cylinder, DiagramBuilder,
                         DirectCollocation,
                         FiniteHorizonLinearQuadraticRegulatorOptions,
                         FramePoseVector, LeafSystem, LeafSystem_,
                         LogVectorOutput,
                         MakeFiniteHorizonLinearQuadraticRegulator,
                         MeshcatVisualizerCpp, MultibodyPlant, Parser,
                         PiecewisePolynomial, Rgba, RigidTransform,
                         RotationMatrix, SceneGraph, Simulator, Solve,
                         StartMeshcat, TemplateSystem, TrajectorySource,
                         namedview,
                         AddMultibodyPlantSceneGraph,
                         Wing, ConstantVectorSource,ExternallyAppliedSpatialForce, BasicVector,
                         SpatialVelocity)

#from underactuated import FindResource
#from underactuated.scenarios import AddShape


In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

## Starship Crossection
![imgage](https://everydayastronaut.com/wp-content/uploads/Articles/Belly_Flop/Belly-Flop-MAIN-Reshoot.00_08_53_06.Still003-800x450.jpg)


In [ ]:
meshcat.Delete()

builder = DiagramBuilder()
plant, scene_graph =AddMultibodyPlantSceneGraph(builder, time_step=0.0)

Parser(plant).AddModelFromFile("ground.urdf", "ground")
Parser(plant).AddModelFromFile("starship/urdf/starship.urdf", "starship")

plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("body"),
                    RigidTransform(RotationMatrix.MakeXRotation(np.pi/3), [0,0,25]))


plant.Finalize()

visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()


context = diagram.CreateDefaultContext()
#context.SetContinuousState(x0)
diagram.Publish(context)
plant_context = plant.GetMyContextFromRoot(context)


simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
visualizer.StartRecording()
simulator.AdvanceTo(3)
visualizer.PublishRecording()

In [ ]:
class SpatialForceConcatinator(LeafSystem):
    def __init__(self, N_inputs):
        LeafSystem.__init__(self)
        self.Input_ports = [self.DeclareAbstractInputPort(f"Spatial_Force_{i}",
                                                          AbstractValue.Make([ExternallyAppliedSpatialForce()])) for i in range(N_inputs)]
        
        self.DeclareAbstractOutputPort("Spatial_Forces", lambda: AbstractValue.Make([ExternallyAppliedSpatialForce() for i in range(N_inputs)]),self.Concatenate)

    def Concatenate(self, context, output):
        out = []
        for port in self.Input_ports:
            out += port.Eval(context)
        output.set_value(out)
        
        
class Barometer(LeafSystem):
    def __init__(self, body_index, P0 = 101325.0, rho0 = 1.2250, T0 = 288.15, L0 = -0.0065, h0 = 0.0, g = 9.80665, M = 0.0289644):
        LeafSystem.__init__(self)
        
        self.body_index = body_index
        self.P0 = P0 #reference pressure (Pa)
        self.rho0 = rho0 #reference density (kg/m^3)
        self.T0 = T0 #reference temperature (K)
        self.L0 = L0 #temperature lapse rate (k/m)
        self.h0 = h0 #height of reference level (m)
        self.g = g #gravitational acceleration (m/s^2)
        self.M = M #molar mass of air (kg/mol)
        
        self.R = 9.31444598 # universal gas constant (J/(mol*K ))
        
        
        self.Pose_port = self.DeclareAbstractInputPort("body_poses_input_port",AbstractValue.Make([RigidTransform()]))
        
        self.DeclareVectorOutputPort("Pressure",BasicVector(1), self.CalcPressure)
        self.DeclareVectorOutputPort("Density",BasicVector(1), self.CalcDensity)
        
    def CalcPressure(self, context, output):
        h = self.Pose_port.Eval(context)[self.body_index].translation()[2]
        
        if self.L0 == 0:
            P = self.P0 * np.exp(-self.g * self.M * (h - self.h0) / (self.R * self.T0))
        else:
            P = self.P0 * (1 + (h - self.h0) * self.L0 / self.T0) ** (-self.g * self.M / (self.R * self.L0))
        output.SetFromVector(np.array([P]))
        
    def CalcDensity(self, context, output):
        h = self.Pose_port.Eval(context)[self.body_index].translation()[2]
        
        if self.L0 == 0:
            rho = self.rho0 * np.exp(-self.g * self.M * (h - self.h0) / (self.R * self.T0))
        else:
            rho = self.rho0 * (self.T0 / (self.T0 + (h - self.h0) * self.L0)) ** (1 + self.g * self.M / (self.R * self.L0))
        output.SetFromVector(np.array([rho]))
            

In [ ]:
meshcat.Delete()

builder = DiagramBuilder()
plant, scene_graph =AddMultibodyPlantSceneGraph(builder, time_step=0.001)

Parser(plant).AddModelFromFile("ground.urdf", "ground")
Parser(plant).AddModelFromFile("starship/urdf/starship.urdf", "starship")
plant.Finalize()

visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

#No wind 
no_wind = builder.AddSystem(ConstantVectorSource([0,0,0]))

barometer = builder.AddSystem(Barometer(body_index = plant.GetFrameByName("body").body().index()))

belly_1 = builder.AddSystem(Wing(body_index = plant.GetFrameByName("body").body().index(),
          X_BodyWing = RigidTransform(),
          surface_area = 395.82, fluid_density = 1.204))

belly_2 = builder.AddSystem(Wing(body_index = plant.GetFrameByName("body").body().index(),
          X_BodyWing = RigidTransform(RotationMatrix.MakeYRotation(np.pi/2)),
          surface_area = 61.164, fluid_density = 1.204))

Leg_R = builder.AddSystem(Wing(body_index = plant.GetFrameByName("Link_Leg_R").body().index(),
          X_BodyWing = RigidTransform(p = [0,-1.88,0]),
          surface_area = 47.859, fluid_density = 1.204))

Leg_L = builder.AddSystem(Wing(body_index = plant.GetFrameByName("Link_Leg_L").body().index(),
          X_BodyWing = RigidTransform(p = [0,1.88,0]),
          surface_area = 47.859, fluid_density = 1.204))

Arm_R = builder.AddSystem(Wing(body_index = plant.GetFrameByName("Link_Arm_R").body().index(),
          X_BodyWing = RigidTransform(p = [0.22,-1.24,0]),
          surface_area = 16.295, fluid_density = 1.204))

Arm_L = builder.AddSystem(Wing(body_index = plant.GetFrameByName("Link_Arm_L").body().index(),
          X_BodyWing = RigidTransform(p = [0.22,1.24,0]),
          surface_area = 16.295, fluid_density = 1.204))

wings = [belly_1, belly_2, Leg_R, Leg_L, Arm_R, Arm_L]

concat = builder.AddSystem(SpatialForceConcatinator(len(wings)))

builder.Connect(plant.get_body_poses_output_port(), barometer.GetInputPort("body_poses_input_port"))

for i, control_surface in enumerate(wings):
    #inputs 
    builder.Connect(plant.get_body_poses_output_port(), control_surface.get_body_poses_input_port())
    builder.Connect(plant.get_body_spatial_velocities_output_port(), control_surface.get_body_spatial_velocities_input_port())

    builder.Connect(no_wind.get_output_port(0), control_surface.get_wind_velocity_input_port())
    builder.Connect(barometer.GetOutputPort("Density"), control_surface.get_fluid_density_input_port())
    
    #output
    builder.Connect(control_surface.get_spatial_force_output_port(), concat.get_input_port(i))
    
#output
builder.Connect(concat.get_output_port(0), plant.get_applied_spatial_force_input_port())


diagram = builder.Build()

simulator = Simulator(diagram)

sim_context = simulator.get_mutable_context()
plant_context = plant.GetMyContextFromRoot(sim_context)

#initial condition
V_WB = SpatialVelocity(w=[0, 0, 0], v=[0,0,-20])
X_WB = RigidTransform(RotationMatrix.MakeYRotation(-np.pi/8),[0, 0, 50])

plant.SetFreeBodyPose(plant_context, plant.GetBodyByName("body"), X_WB)
plant.SetFreeBodySpatialVelocity(context=plant_context, body=plant.GetBodyByName("body"), V_WB=V_WB)


simulator.set_target_realtime_rate(1.0)
visualizer.StartRecording()
simulator.AdvanceTo(5)
visualizer.PublishRecording()